In [21]:
from torch.utils.data import Dataset
import os
import torch
import torchaudio
import pandas as pd
import numpy as np
import sys

sys.path.append('../utils')
import audio_utils


In [24]:
aud00 = torch.rand([1,1323000])
aud01 = torch.rand([1,1323001])

audio_utils.get_log_melSpectrogram(aud00).shape, audio_utils.get_log_melSpectrogram(aud01).shape

(torch.Size([1, 40, 4135]), torch.Size([1, 40, 4135]))

In [8]:
class scraperDataset(Dataset):

    '''datadir = '../../audioData/sythenticSoundscenes/test/'
    csv = 'scrapper_dataset.csv'''

    def __init__(self, dataset_csv, data_dir, only_scene=False):

        self.dataset_csv = dataset_csv
        self.data_directory = data_dir
        self.only_scene = only_scene
        self.dataframe = pd.read_csv(dataset_csv)

        self.audio_files = self.dataframe['audio_fileNames']
        self.label_files = self.dataframe['label_fileNames']
        self.scene_labels = self.dataframe['acoustic_scene_label']
        self.events_label_list = self.dataframe['events_label_list']

    def __len__(self):
        return len(self.audio_files)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Load Audio file
        audio_file = os.path.join(self.data_directory, self.audio_files[idx])
        audio_data, sr = torchaudio.load(audio_file)

        if self.only_scene:
            sample = {'audio':audio_data, 'scene_label':self.scene_labels[idx]}
        else:
            sample = {'audio':audio_data, 'scene_label':self.scene_labels[idx], 'event_list':self.events_label_list[idx]}
        
        return sample


In [10]:
class TUT18_Dataset(Dataset):

    '''datadir = '../../audioData/TUTUrban2018/developmentDataset/TUT-urban-acoustic-scenes-2018-development/'
    csv = 'TUT18_train.csv'''

    def __init__(self, dataset_csv, data_dir):

        self.dataset_csv = dataset_csv
        self.data_directory = data_dir
        self.dataframe = pd.read_csv(dataset_csv)

        self.audio_files = self.dataframe['files']
        self.scene_labels = self.dataframe['labels']

    def __len__(self):
        return len(self.audio_files)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        audio_file = os.path.join(self.data_directory, self.audio_files[idx])
        audio_data, sr = torchaudio.load(audio_file)

        sample = {'audio':audio_data, 'scene_label':self.scene_labels[idx]}
        return sample

In [6]:
datadir = '../../audioData/sythenticSoundscenes/test/'
#datadir = '../../audioData/TUTUrban2018/developmentDataset/TUT-urban-acoustic-scenes-2018-development/'
csv = 'scrapper_test_dataset.csv'
#csv = 'TUT18_train.csv'
scrapper_df = pd.read_csv(csv)

#dataset = scraperDataset(csv, datadir)
#dataset = TUT18_Dataset(csv, datadir)

#f10 = [dataset[i] for i in range(10)]
import re
pattern = r'[^a-zA-Z0-9\s]'
event_labels_set = set()

for event_list in scrapper_df['events_label_list']:
    event_list = re.sub(pattern, '', event_list)
    event_labels_set.update(event_list.split(' '))

len(event_labels_set)

32

In [20]:
labels_list = []
for event_list in scrapper_df['events_label_list']:
    event_list = re.sub(pattern, '', event_list)
    labels_list.append(event_list.split(' '))

from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(classes=list(event_labels_set))
one_hot_labes = mlb.fit_transform(labels_list)

len(one_hot_labes[0]),one_hot_labes[0]

(32,
 array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1]))

In [12]:
f10[2]['scene_label']

'airport'